# ESP32 File System

The Huzzah32 has a small flash chip attached that is used for storing the MicroPython interpreter and user code and data. Many of the basic file operations are available to list the contents of the flash chip and copy files to and from the host computer. Below is a summary.

### Directory Listing

In [1]:
%connect "/dev/cu.SLAB_USBtoUART"

import os

print(os.listdir('/'))

Connected on /dev/cu.SLAB_USBtoUART
['remote', 'boot.py', 'code', 'liblib', 'lib']


`remote` is a virtual folder that points back to the files on the host. E.g.

In [1]:
print(os.listdir('/remote/code/lib'))

['plotclient.py', 'logging.py', 'board.py', '.DS_Store', 'functools.py', 'writer.py', 'mpu9250.py', 'upip.py', 'imu.py', 'urequests.py', 'freesans20.py', 'vector3d.py', 'ina219.py', 'oledfeather.py', 'ads1x15.py', 'mqttclient.py']


shows the names of files in the `lib` folder on the host computer.

### Copy Files

Use the `copy` command to copy a single file to the esp32 (or back to the host). E.g.

In [1]:
Util.mkdirs('/lib/')    
Util.copy('/remote/code/lib/board.py', '/lib/board.py')

Copying: /remote/code/lib/board.py -> /lib/board.py (654 bytes)


creates a folder `lib` on the esp32 and copies the file `board.py` from the host.

### Synchronize Folder

To copy an entire folder from the host to the esp32, use the `sync` command:

In [1]:
Util.sync('/remote/code/lib', '/', include='.*\.py')

Copying: /remote/code/lib/plotclient.py -> /lib/plotclient.py (1229 bytes)
Copying: /remote/code/lib/logging.py -> /lib/logging.py (1646 bytes)
Up to date: /remote/code/lib/board.py
Copying: /remote/code/lib/functools.py -> /lib/functools.py (591 bytes)
Copying: /remote/code/lib/writer.py -> /lib/writer.py (5142 bytes)
Copying: /remote/code/lib/mpu9250.py -> /lib/mpu9250.py (8945 bytes)
Copying: /remote/code/lib/upip.py -> /lib/upip.py (7894 bytes)
Copying: /remote/code/lib/imu.py -> /lib/imu.py (14331 bytes)
Copying: /remote/code/lib/urequests.py -> /lib/urequests.py (2801 bytes)
Copying: /remote/code/lib/freesans20.py -> /lib/freesans20.py (16985 bytes)
Copying: /remote/code/lib/vector3d.py -> /lib/vector3d.py (4784 bytes)
Copying: /remote/code/lib/ina219.py -> /lib/ina219.py (17314 bytes)
Copying: /remote/code/lib/oledfeather.py -> /lib/oledfeather.py (634 bytes)
Copying: /remote/code/lib/ads1x15.py -> /lib/ads1x15.py (8289 bytes)
Copying: /remote/code/lib/mqttclient.py -> /lib/mqtt

The `sync` command is clever: it copies only files that have changed or are not already present on the esp32.

### List all files on esp32

Use the code below to list all files on the esp32:

In [1]:
for path in Util.walk('/'):
    if path is '//remote': continue
    print(path[1:])

/lib/board.py
/lib/plotclient.py
/lib/logging.py
/lib/functools.py
/lib/writer.py
/lib/mpu9250.py
/lib/upip.py
/lib/imu.py
/lib/urequests.py
/lib/freesans20.py
/lib/vector3d.py
/lib/ina219.py
/lib/oledfeather.py
/lib/ads1x15.py
/lib/mqttclient.py


### Delete all files on esp32

The code below removes all files on the esp32. You do not normally need this.

In [1]:
import os

for path in Util.walk('/'):
    if path is '//remote':
        continue
    print("remove", path)
    os.remove(path)

# Libraries and `boot.py`

Now that you have copied several library files to the esp32, you can use them. The code below imports the `board` library to refer to the LED on the eps32 by name, rather than number:

In [2]:
from machine import Pin
from board import LED
import sys, time

sys.path.append('/lib')

led = Pin(LED, mode=Pin.OUT)

for i in range(9):
    led.value(not led.value())
    time.sleep(0.5)

The statement `sys.path.append('/lib')` tells Python to search the `/lib` folder when importing libraries. Since most programs use libraries saved in `/lib` (you can create additional folders and add them to the search path), it is conventient to execute the code that sets up the path automatically.

Whenever the esp32 starts (after power up or pressing the reset button), it executes the file `boot.py` followed by `main.py` if it exists. Hence placing the following code

```python
import sys
sys.path.append('/lib')
```

into a file named `boot.py` and copying it to the esp32 alleviates the need to include these statements in every program. Later we will add additional functionality to `boot.py`, e.g. for connecting to the wireless network.